In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install folium==0.2.1
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip install -q tflite-model-maker
!pip install -q tflite-support

import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.6 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=ed7e6ec8a69cef0de82bd97ed9201ac4a0bdccd22caf7fafbbfc2bb5c6c28117
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.12.1.post1
    Uninstalling folium-0.12.1.post1:
      Successfully uninstalled folium-0.12.1.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.5 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simpl

In [2]:
!unzip -q /content/drive/MyDrive/object_detector/training_set.zip

In [8]:
from pydoc import importfile
classes = importfile("/content/training_set/classes.py")

train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/training_set/train',
    '/content/training_set/train',
    classes.CLASSES
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/training_set/validate',
    '/content/training_set/validate',
    classes.CLASSES
)

In [9]:
spec = model_spec.get('efficientdet_lite0')
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
15/15 [==============================] - 44s 644ms/step - det_loss: 1.6611 - cls_loss: 1.1040 - box_loss: 0.0111 - reg_l2_loss: 0.0630 - loss: 1.7241 - learning_rate: 0.0065 - gradient_norm: 1.9800 - val_det_loss: 1.5662 - val_cls_loss: 1.0723 - val_box_loss: 0.0099 - val_reg_l2_loss: 0.0630 - val_loss: 1.6292
Epoch 2/20
15/15 [==============================] - 7s 488ms/step - det_loss: 1.4134 - cls_loss: 0.9842 - box_loss: 0.0086 - reg_l2_loss: 0.0630 - loss: 1.4763 - learning_rate: 0.0049 - gradient_norm: 3.4590 - val_det_loss: 1.1898 - val_cls_loss: 0.8023 - val_box_loss: 0.0077 - val_reg_l2_loss: 0.0630 - val_loss: 1.2528
Epoch 3/20
15/15 [==============================] - 7s 490ms/step - det_loss: 1.1200 - cls_loss: 0.7491 - box_loss: 0.0074 - reg_l2_loss: 0.0630 - loss: 1.1830 - learning_rate: 0.0048 - gradient_norm: 2.8600 - val_det_loss: 0.9086 - val_cls_loss: 0.5883 - val_box_loss: 0.0064 - val_reg_l2_loss: 0.0630 - val_loss: 0.9716
Epoch 4/20
15/15 [===============

In [10]:
model.evaluate(val_data)

1/1 [==============================] - 7s 7s/step



{'AP': 0.8135565,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.8135565,
 'ARmax1': 0.8361111,
 'ARmax10': 0.8423611,
 'ARmax100': 0.8423611,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.8423611,
 'AP_/android': 0.82260257,
 'AP_/pig_android': 0.8045105}

In [11]:
model.export(export_dir='.', tflite_filename='detector.tflite')

In [12]:
model.evaluate_tflite('detector.tflite', val_data)

10/10 [==============================] - 26s 2s/step



{'AP': 0.79163367,
 'AP50': 1.0,
 'AP75': 1.0,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.79163367,
 'ARmax1': 0.81805557,
 'ARmax10': 0.81805557,
 'ARmax100': 0.81805557,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.81805557,
 'AP_/android': 0.80594057,
 'AP_/pig_android': 0.7773267}

In [13]:
!mv detector.tflite /content/drive/MyDrive/object_detector/